In [1]:
import numpy as np
import pandas as  pd
import matplotlib.pyplot as plt
from keras.utils import image_dataset_from_directory
import os
import cv2

In [2]:
df_train="/kaggle/input/image-matching-challenge-2024/train/"

In [3]:
c=cv2.imread("/kaggle/input/image-matching-challenge-2024/test/church/images/00030.png")
c.shape

(1024, 768, 3)

In [4]:
# image_path=[]
# label=[]
# for cur,dir,filename in os.walk(df_train):
#     for file in filename:
#         if file.endswith((".png",".jpg",".jpeg")):
#             data=os.path.join(cur,file)
#             cv=cv2.imread(data)
#             cv_resize=cv2.resize(cv,(1024, 768))
#             image_path.append(data)
#             label.append(os.path.basename(cur))

In [5]:
# print(len(image_path))
# len(label)

In [6]:
# np.unique(np.array(label))

In [7]:
training_data=image_dataset_from_directory(
    df_train,
    seed=123,
    label_mode='categorical',
    validation_split=0.2,
    shuffle=True,
    subset="training",
    batch_size=32,
    image_size=(1024, 768),
)
validation_data=image_dataset_from_directory(
    df_train,
    seed=123,
    label_mode='categorical',
    validation_split=0.2,
    shuffle=True,
    subset="validation",
    batch_size=32,
    image_size=(1024, 768),
)

Found 2203 files belonging to 7 classes.
Using 1763 files for training.
Found 2203 files belonging to 7 classes.
Using 440 files for validation.


In [8]:
for img,label in training_data.take(1):
    print(img.shape,label.shape)
    print(label)

(32, 1024, 768, 3) (32, 7)
tf.Tensor(
[[0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]], shape=(32, 7), dtype=float32)


# # Deep learning

In [9]:
from keras.layers import Dense,Flatten,Input,Conv2D,MaxPooling2D,BatchNormalization
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras.models import Model

In [12]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1)

# # M

In [13]:
inputs=Input(shape=(1024, 768, 3))
c=Conv2D(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
c=Conv2D(15,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(c)
m=MaxPooling2D(2,2)(c)
b=BatchNormalization()(m)
f=Flatten()(b)
d=Dense(32,activation="relu")(f)
outputs=Dense(7,activation="softmax")(d)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=["accuracy"])

In [31]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1024, 768, 3)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 1024, 768, 32)  │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 1024, 768, 15)  │         4,335 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 512, 384, 15)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 512, 384, 15)   │            60 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2949120)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │    94,371,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 283,132,124 (1.05 GB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 94,377,394 (360.02 MB)

 Optimizer params: 188,754,730 (720.04 MB)

In [15]:
model.fit(training_data,epochs=50,validation_data=validation_data,callbacks=[early_stopping])

Epoch 1/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 723ms/step - accuracy: 0.5702 - loss: 225.5904

2024-08-26 11:20:37.584890: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng11{k2=2,k3=0} for conv (f32[24,15,1024,768]{3,2,1,0}, u8[0]{0}) custom-call(f32[24,32,1024,768]{3,2,1,0}, f32[15,32,3,3]{3,2,1,0}, f32[15]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-08-26 11:20:37.760001: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.175212179s
Trying algorithm eng11{k2=2,k3=0} for conv (f32[24,15,1024,768]{3,2,1,0}, u8[0]{0}) custom-call(f32[24,32,1024,768]{3,2,1,0}, f32[15,32,3,3]{3,2,1,0}, f32[15]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"op

56/56 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step - accuracy: 0.5726 - loss: 223.8727 - val_accuracy: 0.3182 - val_loss: 904.5330
Epoch 2/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 49s 869ms/step - accuracy: 0.8964 - loss: 13.9776 - val_accuracy: 0.4886 - val_loss: 130.4811
Epoch 3/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 49s 863ms/step - accuracy: 0.8096 - loss: 11.3738 - val_accuracy: 0.4045 - val_loss: 38.1569
Epoch 4/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 49s 862ms/step - accuracy: 0.7965 - loss: 4.9411 - val_accuracy: 0.5705 - val_loss: 74.4946
Epoch 5/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 49s 859ms/step - accuracy: 0.8298 - loss: 2.9153 - val_accuracy: 0.7227 - val_loss: 8.4611
Epoch 6/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 49s 863ms/step - accuracy: 0.8397 - loss: 4.8314 - val_accuracy: 0.6568 - val_loss: 12.0317
Epoch 7/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 49s 858ms/step - accuracy: 0.8317 - loss: 3.1611 - val_accuracy: 0.7659 - val_loss: 7.6083
Epoch 8/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 49s 876ms/step - accuracy: 0.8328 - loss: 3.4788 - val_accurac

In [18]:
loss,accuracy=model.evaluate(validation_data)

14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 582ms/step - accuracy: 0.7768 - loss: 7.0688


In [32]:
for layer in model.layers:
    layer.trainable=False
model_data=model.layers[-4].output

# # M2

In [37]:
# inputs=Input(shape=(1024, 768, 3))
c=Conv2D(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(model_data)
c=Conv2D(64,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(c)
m=MaxPooling2D(2,2)(c)
b=BatchNormalization()(m)

c=Conv2D(128,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(b)
c=Conv2D(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(c)
m=MaxPooling2D(2,2)(c)
b=BatchNormalization()(m)

f=Flatten()(b)
d=Dense(32,activation="relu")(f)
outputs2=Dense(7,activation="softmax")(d)
model2=Model(inputs=model.input,outputs=outputs2)
model2.compile(optimizer='adam',loss='categorical_crossentropy',metrics=["accuracy"])

In [38]:
model2.fit(training_data,epochs=50,validation_data=validation_data,callbacks=[early_stopping])

Epoch 1/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 56s 902ms/step - accuracy: 0.6154 - loss: 15.4157 - val_accuracy: 0.5773 - val_loss: 6.8313
Epoch 2/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 49s 875ms/step - accuracy: 0.8436 - loss: 5.8830 - val_accuracy: 0.7909 - val_loss: 5.7756
Epoch 3/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 49s 865ms/step - accuracy: 0.8795 - loss: 5.3582 - val_accuracy: 0.8818 - val_loss: 5.2798
Epoch 4/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 49s 871ms/step - accuracy: 0.9120 - loss: 5.0581 - val_accuracy: 0.8432 - val_loss: 5.1757
Epoch 5/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 50s 871ms/step - accuracy: 0.8925 - loss: 4.9562 - val_accuracy: 0.8091 - val_loss: 5.5929
Epoch 6/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 49s 861ms/step - accuracy: 0.8657 - loss: 4.7594 - val_accuracy: 0.8500 - val_loss: 4.7217
Epoch 7/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 49s 871ms/step - accuracy: 0.8839 - loss: 4.4619 - val_accuracy: 0.8591 - val_loss: 5.7096
Epoch 8/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 49s 858ms/step - accuracy: 0.8521 - loss: 4.5246 - val_acc

In [39]:
loss,accuracy=model2.evaluate(validation_data)

14/14 ━━━━━━━━━━━━━━━━━━━━ 9s 587ms/step - accuracy: 0.8390 - loss: 1.6515


In [40]:
model2.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1024, 768, 3)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 1024, 768, 32)  │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 1024, 768, 15)  │         4,335 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 512, 384, 15)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 512, 384, 15)   │            60 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 512, 384, 32)   │         4,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 512, 384, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 256, 192, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 256, 192, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 256, 192, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 256, 192, 32)   │        36,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 128, 96, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 128, 96, 32)    │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 393216)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 32)             │    12,582,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 7)              │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,156,386 (145.56 MB)

 Trainable params: 12,716,967 (48.51 MB)

 Non-trainable params: 5,483 (21.42 KB)

 Optimizer params: 25,433,936 (97.02 MB)

In [42]:
for layer in model2.layers:
    layer.trainable=False
model2_data=model2.layers[-4].output

# # M3

In [43]:
# inputs=Input(shape=(1024, 768, 3))
c=Conv2D(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(model2_data)
c=Conv2D(64,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(c)
m=MaxPooling2D(2,2)(c)
b=BatchNormalization()(m)

c=Conv2D(128,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(b)
c=Conv2D(320,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(c)
m=MaxPooling2D(2,2)(c)
b=BatchNormalization()(m)

c=Conv2D(128,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(b)
c=Conv2D(320,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(c)
m=MaxPooling2D(2,2)(c)
b=BatchNormalization()(m)

f=Flatten()(b)
d=Dense(32,activation="relu")(f)
outputs3=Dense(7,activation="softmax")(d)
model3=Model(inputs=model.input,outputs=outputs3)
model3.compile(optimizer='adam',loss='categorical_crossentropy',metrics=["accuracy"])

In [45]:
model3.fit(training_data,epochs=50,validation_data=validation_data,callbacks=[early_stopping])

Epoch 1/50


2024-08-26 12:14:17.990818: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng4{k11=1} for conv (f32[32,32,128,96]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,64,128,96]{3,2,1,0}, f32[64,32,3,3]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-08-26 12:14:18.187284: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.196596097s
Trying algorithm eng4{k11=1} for conv (f32[32,32,128,96]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,64,128,96]{3,2,1,0}, f32[64,32,3,3]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn

56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 684ms/step - accuracy: 0.7215 - loss: 20.5288

I0000 00:00:1724674502.334240     110 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion_5', 176 bytes spill stores, 176 bytes spill loads



56/56 ━━━━━━━━━━━━━━━━━━━━ 72s 919ms/step - accuracy: 0.7232 - loss: 20.5068 - val_accuracy: 0.1523 - val_loss: 50.2786
Epoch 2/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 42s 743ms/step - accuracy: 0.8714 - loss: 15.1188 - val_accuracy: 0.7182 - val_loss: 13.5351
Epoch 3/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 41s 730ms/step - accuracy: 0.9118 - loss: 10.7913 - val_accuracy: 0.8659 - val_loss: 8.7534
Epoch 4/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 41s 730ms/step - accuracy: 0.8775 - loss: 7.8363 - val_accuracy: 0.8705 - val_loss: 6.2461
Epoch 5/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 41s 727ms/step - accuracy: 0.9084 - loss: 5.4998 - val_accuracy: 0.8136 - val_loss: 5.2135
Epoch 5: early stopping


In [46]:
loss,accuracy=model3.evaluate(validation_data)

14/14 ━━━━━━━━━━━━━━━━━━━━ 9s 547ms/step - accuracy: 0.8346 - loss: 5.0675
